## 서울교통공사 지하철역 시간대별 지나간 지하철 대수 전처리
- https://data.seoul.go.kr/dataList/OA-101/A/1/datasetView.do

### 데이터 불러오기

In [1]:
import requests

# 실제 서비스 키
SERVICE_KEY = '6c41706a496e617931394343774c44'
# 전체 지하철 역 코드를 저장할 리스트 초기화
subway_codes = []

# 전체 지하철 역 코드 목록을 가져오기 위한 URL
station_list_url = f'http://openAPI.seoul.go.kr:8088/{SERVICE_KEY}/json/SearchInfoBySubwayNameService/1/1000/'

# API를 사용하여 전체 지하철 역 코드 목록을 가져오기
req = requests.get(station_list_url)
print(req)
print(req.text)

<Response [200]>
{"SearchInfoBySubwayNameService":{"list_total_count":777,"RESULT":{"CODE":"INFO-000","MESSAGE":"정상 처리되었습니다"},"row":[{"STATION_CD":"0244","STATION_NM":"용답","LINE_NUM":"02호선","FR_CODE":"211-1"},{"STATION_CD":"0245","STATION_NM":"신답","LINE_NUM":"02호선","FR_CODE":"211-2"},{"STATION_CD":"0250","STATION_NM":"용두","LINE_NUM":"02호선","FR_CODE":"211-3"},{"STATION_CD":"0336","STATION_NM":"학여울","LINE_NUM":"03호선","FR_CODE":"346"},{"STATION_CD":"0428","STATION_NM":"삼각지","LINE_NUM":"04호선","FR_CODE":"428"},{"STATION_CD":"0429","STATION_NM":"신용산","LINE_NUM":"04호선","FR_CODE":"429"},{"STATION_CD":"1008","STATION_NM":"이촌","LINE_NUM":"경의선","FR_CODE":"K111"},{"STATION_CD":"1014","STATION_NM":"청량리","LINE_NUM":"경의선","FR_CODE":"K117"},{"STATION_CD":"103C","STATION_NM":"대곡","LINE_NUM":"서해선","FR_CODE":"836"},{"STATION_CD":"104C","STATION_NM":"능곡","LINE_NUM":"서해선","FR_CODE":"837"},{"STATION_CD":"105C","STATION_NM":"곡산","LINE_NUM":"서해선","FR_CODE":"835"},{"STATION_CD":"106C","STATION_NM":"백마","LINE_N

In [2]:
if req.status_code == 200:
    data = req.json()
    if 'SearchInfoBySubwayNameService' in data:
        for item in data['SearchInfoBySubwayNameService']['row']:
            subway_code = item['STATION_CD']  # 역 코드 가져오기
            subway_codes.append(subway_code)
    else:
        print("데이터에 'SearchInfoBySubwayNameService'가 없습니다.")
else:
    print("역 코드 목록을 가져오는 데 실패했습니다.")

In [3]:
# 각 지하철 역 코드에 대한 데이터 수집
all_data = []  # 모든 역의 데이터를 저장할 리스트 초기화

# 모든 요일과 상하행에 대한 조합에 대해 데이터 수집
for subway_code in subway_codes:
    for week_tag in ['1', '2', '3']:  # 모든 요일에 대한 조합
        for inout_tag in ['1', '2']:  # 상하행에 대한 조합
            url = f'http://openAPI.seoul.go.kr:8088/{SERVICE_KEY}/json/SearchSTNTimeTableByIDService/1/1000/{subway_code}/{week_tag}/{inout_tag}/'
            req = requests.get(url)

            if req.status_code == 200:
                data = req.json()
                all_data.append(data)  # 각 역의 데이터를 all_data 리스트에 추가
            else:
                print(f"역 코드 {subway_code}, 요일 {week_tag}, 상하행 {inout_tag}에 대한 데이터를 가져오는 데 실패했습니다.")

# all_data 리스트에는 각 역의 데이터가 저장되어 있습니다.

In [4]:
print(subway_codes)

['0244', '0245', '0250', '0336', '0428', '0429', '1008', '1014', '103C', '104C', '105C', '106C', '107C', '108C', '0412', '0417', '102C', '1030', '1217', '1218', '1286', '1407', '1450', '1452', '1455', '1456', '1504', '3227', '0310', '0319', '0339', '0415', '0425', '100C', '1013', '1264', '1265', '1279', '1408', '1458', '1502', '0248', '0320', '0327', '1005', '1031', '1204', '1510', '1703', '1720', '1834', '1864', '1875', '3753', '4306', '0216', '0249', '0311', '0315', '0323', '0328', '0337', '0421', '1319', '1835', '1856', '1878', '1882', '3117', '3135', '3223', '3224', '3225', '3226', '0233', '0237', '0405', '0430', '1015', '1032', '1208', '1219', '1402', '1885', '1886', '1901', '1981', '2620', '2640', '2724', '2816', '2817', '0338', '0416', '0427', '1201', '1266', '1268', '1804', '1953', '1954', '2530', '2535', '2557', '2558', '2815', '3754', '3755', '3756', '3757', '3758', '3759', '0316', '0329', '0334', '0341', '1009', '1203', '1214', '1404', '1812', '1854', '1866', '1870', '1871',

### 데이터 프레임 만들기

In [5]:
import pandas as pd

# all_data 리스트에 있는 모든 데이터를 하나의 데이터프레임으로 합치기
all_data_frames = []
for data in all_data:
    if 'SearchSTNTimeTableByIDService' in data:
        df = pd.DataFrame(data['SearchSTNTimeTableByIDService']['row'])
        all_data_frames.append(df)

# 모든 데이터프레임을 하나로 합치기
all_df = pd.concat(all_data_frames, ignore_index=True)


In [6]:
all_df.head(50)

,LINE_NUM,FR_CODE,STATION_CD,STATION_NM,TRAIN_NO,ARRIVETIME,LEFTTIME,ORIGINSTATION,DESTSTATION,SUBWAYSNAME,SUBWAYENAME,WEEK_TAG,INOUT_TAG,FL_FLAG,DESTSTATION2,EXPRESS_YN,BRANCH_LINE
0,02호선,211-1,0244,용답,1502,05:35:30,05:36:00,0246,0211,신설동,성수,1,1,,,G,
1,02호선,211-1,0244,용답,1504,05:52:30,05:53:00,0246,0211,신설동,성수,1,1,,,G,
2,02호선,211-1,0244,용답,1506,06:07:30,06:08:00,0246,0211,신설동,성수,1,1,,,G,
3,02호선,211-1,0244,용답,1508,06:22:30,06:23:00,0246,0211,신설동,성수,1,1,,,G,
4,02호선,211-1,0244,용답,1510,06:38:30,06:39:00,0246,0211,신설동,성수,1,1,,,G,
5,02호선,211-1,0244,용답,1512,06:50:30,06:51:00,0246,0211,신설동,성수,1,1,,,G,
6,02호선,211-1,0244,용답,1514,07:00:30,07:01:00,0246,0211,신설동,성수,1,1,,,G,
7,02호선,211-1,0244,용답,1516,07:11:30,07:12:00,0246,0211,신설동,성수,1,1,,,G,
8,02호선,211-1,0244,용답,1518,07:21:30,07:22:00,0246,0211,신설동,성수,1,1,,,G,
9,02호선,211-1,0244,용답,1520,07:28:30,07:29:00,0246,0211,신설동,성수,1,1,,,G,


In [7]:
all_df.LINE_NUM.value_counts()

01호선    81653
02호선    61845
05호선    49965
07호선    49441
04호선    47846
03호선    42793
09호선    33286
06호선    30501
08호선    14197
Name: LINE_NUM, dtype: int64

In [8]:
all_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 411527 entries, 0 to 411526
Data columns (total 17 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   LINE_NUM       411527 non-null  object
 1   FR_CODE        411527 non-null  object
 2   STATION_CD     411527 non-null  object
 3   STATION_NM     411527 non-null  object
 4   TRAIN_NO       411527 non-null  object
 5   ARRIVETIME     411527 non-null  object
 6   LEFTTIME       411527 non-null  object
 7   ORIGINSTATION  411527 non-null  object
 8   DESTSTATION    411527 non-null  object
 9   SUBWAYSNAME    411527 non-null  object
 10  SUBWAYENAME    411527 non-null  object
 11  WEEK_TAG       411527 non-null  object
 12  INOUT_TAG      411527 non-null  object
 13  FL_FLAG        411527 non-null  object
 14  DESTSTATION2   411527 non-null  object
 15  EXPRESS_YN     411527 non-null  object
 16  BRANCH_LINE    411527 non-null  object
dtypes: object(17)
memory usage: 53.4+ MB


In [9]:
# 요일과 상하행 숫자를 글자로 매핑하는 딕셔너리 생성
week_mapping = {
    '1': '평일',
    '2': '토요일',
    '3': '휴일/일요일'
}

inout_mapping = {
    '1': '상행',
    '2': '하행'
}

In [11]:
all_df.to_csv('지하철 열차시간표.csv')

### 시간대별 지하철 수 계산 및 새로운 데이터프레임 처리

In [12]:
unique_stations = all_df['STATION_CD'].unique()
unique_lines = all_df['LINE_NUM'].unique()
unique_week_tags = all_df['WEEK_TAG'].unique()
unique_inout_tags = all_df['INOUT_TAG'].unique()

# 필요한 시간대 컬럼 생성
time_columns = [f'{i:02d}시~{i+1:02d}시' for i in range(5, 25)]

# 결과 데이터프레임 초기화
result_df = pd.DataFrame(columns=['STATION_CD', 'STATION_NM', 'LINE_NUM', '요일', '상하행'] + time_columns)

# 각 역별로 데이터 처리
for station_cd in unique_stations:
    station_name = all_df[all_df['STATION_CD'] == station_cd]['STATION_NM'].iloc[0]
    for line_num in unique_lines:
        for week_tag in unique_week_tags:
            for inout_tag in unique_inout_tags:
                filtered_df = all_df[(all_df['STATION_CD'] == station_cd) &
                                     (all_df['LINE_NUM'] == line_num) &
                                     (all_df['WEEK_TAG'] == week_tag) &
                                     (all_df['INOUT_TAG'] == inout_tag)]

                # 시간대별 열차 수를 저장할 리스트 초기화
                train_counts = [0] * len(time_columns)

                for index, row in filtered_df.iterrows():
                    # hh:mm:ss 중 h만 가져오기
                    arr_time = int(row['ARRIVETIME'].split(':')[0])
                    # 첫 차: 5시 -> 0 index 부터 train_counts 에 저장
                    if 5 <= arr_time < 25:
                        train_counts[arr_time - 5] += 1

                # 모든 시간대가 0인 행은 추가하지 않음
                if all(count == 0 for count in train_counts):
                    continue

                # 결과 데이터프레임에 행 추가
                temp_df = pd.DataFrame({
                    'STATION_CD': [station_cd],
                    'STATION_NM': [station_name],
                    'LINE_NUM': [line_num],
                    '요일': [week_mapping[str(week_tag)]],
                    '상하행': [inout_mapping[str(inout_tag)]],
                    **{time_col: count for time_col, count in zip(time_columns, train_counts)}})

                result_df = pd.concat([result_df, temp_df], ignore_index=True)

In [15]:
# 결과 출력
result_df

,STATION_CD,STATION_NM,LINE_NUM,요일,상하행,05시~06시,06시~07시,07시~08시,08시~09시,09시~10시,...,15시~16시,16시~17시,17시~18시,18시~19시,19시~20시,20시~21시,21시~22시,22시~23시,23시~24시,24시~25시
0,0244,용답,02호선,평일,상행,2,4,8,7,5,...,5,6,6,7,7,6,5,6,5,4
1,0244,용답,02호선,평일,하행,2,4,7,8,5,...,5,6,6,7,7,5,6,6,5,4
2,0244,용답,02호선,토요일,상행,2,4,5,6,6,...,6,6,6,6,6,5,6,5,6,0
3,0244,용답,02호선,토요일,하행,2,4,5,6,6,...,6,6,6,6,5,6,6,5,6,0
4,0244,용답,02호선,휴일/일요일,상행,2,4,5,6,6,...,6,6,6,6,6,5,6,5,6,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2611,0202,을지로입구,02호선,평일,하행,2,10,12,20,20,...,11,14,15,16,16,10,12,7,9,5
2612,0202,을지로입구,02호선,토요일,상행,3,10,11,11,11,...,11,11,12,11,11,11,9,8,5,1
2613,0202,을지로입구,02호선,토요일,하행,2,10,10,10,11,...,11,12,12,11,11,10,10,9,6,1
2614,0202,을지로입구,02호선,휴일/일요일,상행,3,10,11,11,11,...,11,11,12,11,11,11,9,8,5,1


In [16]:
result_df.to_csv('지하철 열차시간표(요일, 상하행별).csv')

In [17]:
result_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2616 entries, 0 to 2615
Data columns (total 25 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   STATION_CD  2616 non-null   object
 1   STATION_NM  2616 non-null   object
 2   LINE_NUM    2616 non-null   object
 3   요일          2616 non-null   object
 4   상하행         2616 non-null   object
 5   05시~06시     2616 non-null   object
 6   06시~07시     2616 non-null   object
 7   07시~08시     2616 non-null   object
 8   08시~09시     2616 non-null   object
 9   09시~10시     2616 non-null   object
 10  10시~11시     2616 non-null   object
 11  11시~12시     2616 non-null   object
 12  12시~13시     2616 non-null   object
 13  13시~14시     2616 non-null   object
 14  14시~15시     2616 non-null   object
 15  15시~16시     2616 non-null   object
 16  16시~17시     2616 non-null   object
 17  17시~18시     2616 non-null   object
 18  18시~19시     2616 non-null   object
 19  19시~20시     2616 non-null   object
 20  20시~21시 

In [18]:
result_df[time_columns].value_counts()

05시~06시  06시~07시  07시~08시  08시~09시  09시~10시  10시~11시  11시~12시  12시~13시  13시~14시  14시~15시  15시~16시  16시~17시  17시~18시  18시~19시  19시~20시  20시~21시  21시~22시  22시~23시  23시~24시  24시~25시
2        4        6        6        6        6        6        6        6        6        6        6        6        6        6        6        6        5        4        0          6
3        9        10       9        10       10       9        10       11       10       10       10       10       10       10       10       9        8        5        0          4
2        5        8        9        10       9        9        9        9        10       9        9        9        9        10       9        9        8        7        1          4
         7        7        7        7        7        7        7        7        7        7        7        7        7        8        6        7        5        5        1          4
         5        8        9        9        9        9        10       9        9   

In [19]:
# 기초 통계량 확인
stats = result_df[time_columns].describe()
stats

,05시~06시,06시~07시,07시~08시,08시~09시,09시~10시,10시~11시,11시~12시,12시~13시,13시~14시,14시~15시,15시~16시,16시~17시,17시~18시,18시~19시,19시~20시,20시~21시,21시~22시,22시~23시,23시~24시,24시~25시
count,2616,2616,2616,2616,2616,2616,2616,2616,2616,2616,2616,2616,2616,2616,2616,2616,2616,2616,2616,2616
unique,10,21,24,28,25,22,20,19,19,19,20,21,24,25,24,23,22,18,16,13
top,2,7,10,9,10,10,9,10,10,7,7,7,7,7,9,9,9,8,5,0
freq,1007,411,423,381,356,415,407,386,356,415,416,389,403,383,358,428,499,490,622,1203


### 호선별로 데이터 분할 및 csv 파일 저장

In [20]:
import os

# 분할된 데이터를 저장할 디렉토리 경로 설정
output_dir = '호선별_시간별_지하철수데이터/'

# 디렉토리가 존재하지 않으면 생성
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# LINE_NUM 값에 따라 데이터 분할 및 저장
line_nums = ['01호선', '02호선', '03호선', '04호선', '05호선', '06호선', '07호선', '08호선']

for line_num in line_nums:
    # 조건에 맞는 데이터 추출
    subset_df = result_df[result_df['LINE_NUM'] == line_num]

    # CSV 파일로 저장 (파일명은 호선별로 구분)
    file_name = f'{line_num}.csv'
    subset_df.to_csv(output_dir + file_name, index=False)

print("데이터 분할 및 저장 완료.")

데이터 분할 및 저장 완료.


In [22]:
line_01 = pd.read_csv('호선별_시간별_지하철수데이터/01호선.csv')
line_01

,STATION_CD,STATION_NM,LINE_NUM,요일,상하행,05시~06시,06시~07시,07시~08시,08시~09시,09시~10시,...,15시~16시,16시~17시,17시~18시,18시~19시,19시~20시,20시~21시,21시~22시,22시~23시,23시~24시,24시~25시
0,1407,온양온천,01호선,평일,상행,2,3,2,2,2,...,2,2,1,3,2,2,2,3,2,0
1,1407,온양온천,01호선,평일,하행,1,3,2,2,2,...,1,2,2,2,3,3,2,2,2,1
2,1407,온양온천,01호선,토요일,상행,2,1,2,2,1,...,2,1,2,2,3,1,2,3,2,0
3,1407,온양온천,01호선,토요일,하행,0,2,2,1,2,...,2,2,2,2,2,2,3,2,2,0
4,1407,온양온천,01호선,휴일/일요일,상행,2,1,2,2,1,...,2,1,2,2,3,1,2,3,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
574,159,동묘앞,01호선,평일,하행,6,10,15,17,13,...,12,12,14,14,14,14,12,12,7,2
575,159,동묘앞,01호선,토요일,상행,4,10,12,13,13,...,13,14,13,14,14,14,12,12,9,2
576,159,동묘앞,01호선,토요일,하행,5,9,12,12,12,...,13,15,12,13,11,13,12,11,8,0
577,159,동묘앞,01호선,휴일/일요일,상행,4,10,12,13,13,...,13,14,13,14,14,14,12,12,9,2
